# 文本分类实例

## Step1 导入相关包

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step2 加载数据

In [9]:
import pandas as pd
from rich import print

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [10]:
data = data.dropna(how="any").reset_index(drop=True)
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7760,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7761,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7762,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7763,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [11]:
len(data)

7765

## Step3 创建Dataset

In [12]:
from torch.utils.data import Dataset


class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]

    def __len__(self):
        return len(self.data)

In [13]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)

('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)

('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)

(
    '宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因
素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分
钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃
很多~',
    1
)

('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)

## Step4 划分数据集

In [14]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [16]:
for i in range(5):
    print(trainset[i])

('在南山区办事的话住这里还算方便，门口往哪个方向的车都有，就是稍微有点吵啊', 1)

(
    '我一般都不愿意写点评的，但这真的是我入住过的五星级酒店中，也是万丽品牌酒店中最糟糕的。房间当然牵强人意，同
行的同事问我这个酒店是不是非常老了，卫浴设备也都是非常陈旧的。天津的配备跟我在苏州住过的万丽酒店简直是无法相提并论 
。另外，我订的还是行政房，结果办手续的速度可以让我吃一顿早饭了，真的是非常痛苦，我们一行四人，最起码一个人办手续花
了15分钟时间。还有早餐简直让我怀疑这是五星级酒店的早餐，也没有鲜榨橙汁，水果也很糟糕，时蔬更加难吃只有一个冬瓜，
我一共就喝了一杯豆浆和一碗粥。',
    0
)

('如果你没有足够的耐心可以忍受差劲的服务，就不要入住了', 0)

('诺覃祥渣,厄珩祥渣,憩苌腔Ⅰ耋剩.', 1)

('所定的房价间并不是水床房，那里水床只有两间，房间的空气并不好，很闷。', 1)

## Step5 创建Dataloader

In [ ]:
!git lfs clone "https://huggingface.co/hfl/rbt3" --include="*.bin"

In [18]:
import torch

tokenizer = AutoTokenizer.from_pretrained("rbt3")


def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length",
                       truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [19]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32,
                         shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64,
                         shuffle=False, collate_fn=collate_func)

In [20]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ..., 2523,  679,  102],
        [ 101, 2347, 5307,  ...,    0,    0,    0],
        ...,
        [ 101, 3193, 7623,  ...,    0,    0,    0],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 6370, 6579,  ..., 1416, 8013,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 0

## Step6 创建模型及优化器

In [21]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./rbt3/")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./rbt3/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [23]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [24]:
train()

ep: 0, global_step: 0, loss: 0.7237740755081177

ep: 0, global_step: 100, loss: 0.34278246760368347

ep: 0, global_step: 200, loss: 0.1869555562734604

ep: 0, acc: 0.8981958627700806

ep: 1, global_step: 300, loss: 0.0897226333618164

ep: 1, global_step: 400, loss: 0.2580520212650299

ep: 1, acc: 0.8865979313850403

ep: 2, global_step: 500, loss: 0.28963184356689453

ep: 2, global_step: 600, loss: 0.17249375581741333

ep: 2, acc: 0.8994845151901245

## Step9 模型预测

In [25]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！

In [26]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [27]:
pipe(sen)

[{'label': '好评！', 'score': 0.9975244402885437}]